In [ ]:
#  loading the required modules
import cv2
import numpy as np
import urllib.request
import tensorflow as tf
import tensorflow_hub as hub


# the runtime initialization will not allocate all memory on the device to avoid out of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_virtual_device_configuration(gpu,
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])


In [ ]:

# input
img = cv2.imread('path_for_img')    #provide specific path of the image
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0    
#The function converts an input image from one color space to another. In case of a transformation to-from RGB color space, the order of the channels should be specified explicitly (RGB or BGR). Note that the default color format in OpenCV is often referred to as RGB but it is actually BGR (the bytes are reversed). So the first byte in a standard (24-bit) color image will be an 8-bit Blue component, the second byte will be Green, and the third byte will be Red. The fourth, fifth, and sixth bytes would then be the second pixel (Blue, then Green, then Red), and so on.



In [ ]:
img.shape

In [ ]:
# as the model is trained for the input image of size 256 * 256 we need to reshape the input image to the same we didnt replace the old image so that the old dimensions gets presreved and the final image can be prossed to the old dimensions...
img_resized = tf.image.resize(img, [256,256], method='bicubic', preserve_aspect_ratio=False)   
img_resized = tf.transpose(img_resized, [2, 0, 1])
img_input = img_resized.numpy()
reshape_img = img_input.reshape(1,3,256,256)
tensor = tf.convert_to_tensor(reshape_img, dtype=tf.float32)

In [ ]:
from tensorflow import keras

In [ ]:
# load model

module = hub.load(handle = "https://tfhub.dev/intel/midas/v2_1_small/1",tags=['serve'])

In [ ]:
print(module.signatures.keys())

In [ ]:
output = module.signatures['serving_default'](tensor)
prediction = output['default'].numpy()
prediction = prediction.reshape(256, 256)

In [ ]:
# output file
prediction = cv2.resize(prediction, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_CUBIC)  # resizing to the original image dim...

depth_min = prediction.min()
depth_max = prediction.max()
img_out = (255 * (prediction - depth_min) / (depth_max - depth_min)).astype("uint8")

cv2.imwrite("output_img_path", img_out)  # specify the exact path of the output img 

 
